In [6]:
import datetime

from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, Rescaling
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import load_img
import tensorflow as tf
import PIL.Image

epochs = 2000

width = 256
height = 256
model_name = 'v1.h5'

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [8]:
def get_new_model(width, height):
    model = Sequential([
        # Add preprocessing layers here
        Rescaling(1./255, input_shape=(width, height, 1)),
        RandomRotation(0.2),
        RandomZoom(0.2, 0.2),
        # Convolutional layers
        Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1'),
        MaxPooling2D(pool_size=(2, 2), name='maxpool1'),
        Conv2D(64, (3, 3), activation='relu', padding='same', name='conv2'),
        MaxPooling2D(pool_size=(2, 2), name='maxpool2'),
        Conv2D(128, (3, 3), activation='relu', padding='same', name='conv3'),
        MaxPooling2D(pool_size=(2, 2), name='maxpool3'),
        Flatten(name='flatten'),
        Dense(128, activation='relu', name='dense1'),
        Dropout(0.5, name='dropout'),
        Dense(3, activation='softmax', name='output')
    ])
    
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [9]:
train_dir = "./data_training/training"
validation_dir = "./data_training/validation"

def train_model(model, batch_size, width, height, epochs, model_name):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        './data_training/training',
        image_size=(width, height),
        batch_size=batch_size,
        label_mode='categorical',
        color_mode='grayscale')

    validation_ds = tf.keras.preprocessing.image_dataset_from_directory(
        './data_training/validation',
        image_size=(width, height),
        batch_size=batch_size,
        label_mode='categorical',
        color_mode='grayscale')

    log_dir = f"logs/{datetime.datetime.now().strftime('%Y%m%d-%H%M%S')}_{model_name}"
    tensorboard = TensorBoard(log_dir=log_dir, histogram_freq=1, write_images=True)
    checkpoint = ModelCheckpoint(f"{model_name}_{{epoch:02d}}.weights.h5", save_weights_only=True, save_freq='epoch')

    history = model.fit(
        train_ds,
        epochs=epochs,
        validation_data=validation_ds,
        callbacks=[tensorboard, checkpoint])

    return model, history

In [10]:
model = get_new_model(width, height)

model, history = train_model(model, batch_size=16, width=width, height=height, epochs=epochs, model_name=model_name)

Found 4320 files belonging to 3 classes.
Found 4037 files belonging to 3 classes.
Epoch 1/2000
270/270 ━━━━━━━━━━━━━━━━━━━━ 19s 66ms/step - accuracy: 0.4972 - loss: 1.0174 - val_accuracy: 0.7830 - val_loss: 0.6040
Epoch 2/2000
270/270 ━━━━━━━━━━━━━━━━━━━━ 17s 65ms/step - accuracy: 0.6087 - loss: 0.6040 - val_accuracy: 0.7776 - val_loss: 0.6369
Epoch 3/2000
270/270 ━━━━━━━━━━━━━━━━━━━━ 17s 65ms/step - accuracy: 0.6004 - loss: 0.5848 - val_accuracy: 0.7830 - val_loss: 0.5653
Epoch 4/2000
270/270 ━━━━━━━━━━━━━━━━━━━━ 17s 64ms/step - accuracy: 0.6052 - loss: 0.5852 - val_accuracy: 0.7830 - val_loss: 0.5791
Epoch 5/2000
270/270 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - accuracy: 0.6105 - loss: 0.5897 - val_accuracy: 0.7830 - val_loss: 0.6052
Epoch 6/2000
270/270 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - accuracy: 0.6307 - loss: 0.5874 - val_accuracy: 0.7830 - val_loss: 0.6313
Epoch 7/2000
270/270 ━━━━━━━━━━━━━━━━━━━━ 17s 63ms/step - accuracy: 0.6226 - loss: 0.5775 - val_accuracy: 0.7828 - val_loss: 0